In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import multiprocessing_functions as defs
import multiprocessing as mp
import itertools
import logging
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)

from services.faces.face_detector import FaceDetector
from services.faces.comparators.yolo_comparator import YoloComparator

In [3]:
dataset_root = "C:\\Users\\Naofel\\Downloads\\VGG-Face2\\exp10\\val"

def get_person_files(dataset_root) -> dict:
    person_files = {}

    for root, dirs, files in os.walk(dataset_root):
        for dir in dirs:
            person_name = dir
            person_dir = os.path.join(root, dir)
            person_files[person_name] = []

            for file in os.listdir(person_dir):
                if file.endswith(".jpg"):
                    person_files[person_name].append(dir + '\\' + file)

    return person_files

identities = get_person_files(dataset_root)

print(identities)

{'n000001': ['n000001\\0001_01.jpg', 'n000001\\0007_01.jpg', 'n000001\\0012_01.jpg', 'n000001\\0015_01.jpg', 'n000001\\0016_01.jpg', 'n000001\\0019_01.jpg', 'n000001\\0045_01.jpg', 'n000001\\0048_01.jpg', 'n000001\\0054_01.jpg', 'n000001\\0062_01.jpg', 'n000001\\0063_01.jpg', 'n000001\\0067_01.jpg', 'n000001\\0068_01.jpg', 'n000001\\0070_01.jpg', 'n000001\\0073_01.jpg', 'n000001\\0079_01.jpg', 'n000001\\0081_01.jpg', 'n000001\\0082_01.jpg', 'n000001\\0096_01.jpg', 'n000001\\0097_01.jpg', 'n000001\\0097_09.jpg', 'n000001\\0098_01.jpg', 'n000001\\0116_01.jpg', 'n000001\\0120_01.jpg', 'n000001\\0123_01.jpg', 'n000001\\0127_01.jpg', 'n000001\\0129_01.jpg', 'n000001\\0130_01.jpg', 'n000001\\0133_01.jpg', 'n000001\\0138_02.jpg', 'n000001\\0145_01.jpg', 'n000001\\0146_01.jpg', 'n000001\\0149_01.jpg', 'n000001\\0152_01.jpg', 'n000001\\0157_02.jpg', 'n000001\\0167_01.jpg', 'n000001\\0171_01.jpg', 'n000001\\0178_01.jpg', 'n000001\\0183_01.jpg', 'n000001\\0186_03.jpg', 'n000001\\0198_01.jpg', 'n0

In [4]:
positives = []
for key, values in identities.items():
    for i in range(0, len(values)-1):
        for j in range(i+1, len(values)):
            positives.append([values[i], values[j]])

positives = pd.DataFrame(positives, columns = ["file_x", "file_y"])
positives["decision"] = "Yes"

positives

,file_x,file_y,decision
0,n000001\0001_01.jpg,n000001\0007_01.jpg,Yes
1,n000001\0001_01.jpg,n000001\0012_01.jpg,Yes
2,n000001\0001_01.jpg,n000001\0015_01.jpg,Yes
3,n000001\0001_01.jpg,n000001\0016_01.jpg,Yes
4,n000001\0001_01.jpg,n000001\0019_01.jpg,Yes
...,...,...,...
18717,n000010\0214_01.jpg,n000010\0275_01.jpg,Yes
18718,n000010\0214_01.jpg,n000010\0276_01.jpg,Yes
18719,n000010\0272_01.jpg,n000010\0275_01.jpg,Yes
18720,n000010\0272_01.jpg,n000010\0276_01.jpg,Yes


In [5]:
samples_list = list(identities.values())

negatives = []
for i in range(0, len(identities) - 1):
    for j in range(i+1, len(identities)):
        cross_product = itertools.product(samples_list[i], samples_list[j])
        cross_product = list(cross_product)

    for cross_sample in cross_product:
        negatives.append([cross_sample[0], cross_sample[1]])

negatives = pd.DataFrame(negatives, columns = ["file_x", "file_y"])
negatives["decision"] = "No"

negatives

,file_x,file_y,decision
0,n000001\0001_01.jpg,n000010\0005_01.jpg,No
1,n000001\0001_01.jpg,n000010\0013_01.jpg,No
2,n000001\0001_01.jpg,n000010\0016_01.jpg,No
3,n000001\0001_01.jpg,n000010\0018_01.jpg,No
4,n000001\0001_01.jpg,n000010\0028_01.jpg,No
...,...,...,...
17435,n000009\0273_01.jpg,n000010\0169_01.jpg,No
17436,n000009\0273_01.jpg,n000010\0214_01.jpg,No
17437,n000009\0273_01.jpg,n000010\0272_01.jpg,No
17438,n000009\0273_01.jpg,n000010\0275_01.jpg,No


In [6]:
df: pd.DataFrame = pd.concat([positives, negatives]).reset_index(drop = True)
df.file_x = df.file_x
df.file_y = df.file_y

df

,file_x,file_y,decision
0,n000001\0001_01.jpg,n000001\0007_01.jpg,Yes
1,n000001\0001_01.jpg,n000001\0012_01.jpg,Yes
2,n000001\0001_01.jpg,n000001\0015_01.jpg,Yes
3,n000001\0001_01.jpg,n000001\0016_01.jpg,Yes
4,n000001\0001_01.jpg,n000001\0019_01.jpg,Yes
...,...,...,...
36157,n000009\0273_01.jpg,n000010\0169_01.jpg,No
36158,n000009\0273_01.jpg,n000010\0214_01.jpg,No
36159,n000009\0273_01.jpg,n000010\0272_01.jpg,No
36160,n000009\0273_01.jpg,n000010\0275_01.jpg,No


In [7]:
if __name__ == '__main__':
    with mp.Pool(initializer=defs.init_worker, initargs=(dataset_root)) as pool:
        logging.info("Pool size: ", pool._processes)
        names = ['bob', 'sue', 'jim', 'joe', 'sally', 'mavis', 'jane', 'bill', 'george', 'sam', 'henry', 'horatio', 'shannon', 'allan', 'younes', 'loann']
        results = []

        for name in names:
            res = pool.apply_async(defs.greet, (name, "HELLO"))
            results.append(res)

        for res in results:
            print(res.get())